#  Introduction to Bioinformatics

So far in this tutorial, we've primarily worked on the problems of cheminformatics. We've been interested in seeing how we can use the techniques of machine learning to make predictions about the properties of molecules. In this tutorial, we're going to shift a bit and see how we can use classical computer science techniques and machine learning to tackle problems in bioinformatics.

For this, we're going to use the venerable [biopython](https://biopython.org/) library to do some basic bioinformatics. A lot of the material in this notebook is adapted from the extensive official [Biopython tutorial]http://biopython.org/DIST/docs/tutorial/Tutorial.html). We strongly recommend checking out the official tutorial after you work through this notebook!

## Colab

This tutorial and the rest in this sequence are designed to be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Introduction_to_Bioinformatics.ipynb)

## Setup

To run DeepChem within Colab, you'll need to run the following cell of installation commands. This will take about 5 minutes to run to completion and install your environment.

In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3457  100  3457    0     0  11516      0 --:--:-- --:--:-- --:--:-- 11523


add /root/miniconda/lib/python3.10/site-packages to PYTHONPATH
INFO:conda_installer:add /root/miniconda/lib/python3.10/site-packages to PYTHONPATH
python version: 3.10.12
INFO:conda_installer:python version: 3.10.12
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
INFO:conda_installer:fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
INFO:conda_installer:done
installing miniconda to /root/miniconda
INFO:conda_installer:installing miniconda to /root/miniconda
done
INFO:conda_installer:done
installing openmm, pdbfixer
INFO:conda_installer:installing openmm, pdbfixer
added conda-forge to channels
INFO:conda_installer:added conda-forge to channels
done
INFO:conda_installer:done
conda packages installation finished!
INFO:conda_installer:conda packages installation finished!


# conda environments:
#
base                     /root/miniconda



In [2]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.4/827.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 46.3 MB/s eta 0:00:00


'2.7.2.dev'

We'll use `pip` to install `biopython`

In [3]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.1 MB/s eta 0:00:00


In [4]:
import Bio
Bio.__version__

'1.81'

In [5]:
from Bio.Seq import Seq
my_seq = Seq("AGTACACATTG")
my_seq

Seq('AGTACACATTG')

The complement() method in Biopython's Seq object returns the complement of a DNA sequence. It replaces each base with its complement according to the Watson-Crick base pairing rules. Adenine (A) is complemented by thymine (T), and guanine (G) is complemented by cytosine (C).

The reverse_complement() method in Biopython's Seq object returns the reverse complement of a DNA sequence. It first reverses the sequence and then replaces each base with its complement according to the Watson-Crick base pairing rules. 

But why is direction important? Many cellular processes occur only along a particular direction. To understand what gives a sense of directionality to a strand of DNA, take a look at the pictures below. Carbon atoms in the backbone of DNA are numbered from 1' to 5' (usually pronounced as "5 prime") in a clockwise direction. One might notice that the strand on the left has the 5' carbon above the 3' carbon in every nucleotide, resulting in a strand starting with a 5' end and ending with a 3' end. The strand on the right runs from the 3' end to the 5' end. As hinted earlier, reading of a DNA strand during replication and transcription only occurs from the 3' end to the 5' end.
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f2/Nukleotid_num.svg/440px-Nukleotid_num.svg.png">
<img src="https://o.quizlet.com/sLgxc-XlkDjlYalnz2Oa-Q_b.jpg">

In [6]:
my_seq.complement()

Seq('TCATGTGTAAC')

In [7]:
my_seq.reverse_complement()

Seq('CAATGTGTACT')

## Parsing Sequence Records

We're going to download a sample `fasta` file from the Biopython tutorial to use in some exercises. This file is a set of hits for a sequence (of lady slipper orcid genes).It is basically the DNA sequences of a subfamily of plants belonging to Orchids(a diverse group of flowering plants) encoding the ribosomal RNA(rRNA).

In [8]:
!wget https://raw.githubusercontent.com/biopython/biopython/master/Doc/examples/ls_orchid.fasta

--2023-08-02 14:46:07--  https://raw.githubusercontent.com/biopython/biopython/master/Doc/examples/ls_orchid.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76480 (75K) [text/plain]
Saving to: ‘ls_orchid.fasta’

ls_orchid.fasta     100%[===================>]  74.69K  --.-KB/s    in 0.01s   

2023-08-02 14:46:07 (5.24 MB/s) - ‘ls_orchid.fasta’ saved [76480/76480]



Let's take a look at what the contents of this file look like:

1.   List item
2.   List item



In [9]:
from Bio import SeqIO
for seq_record in SeqIO.parse('ls_orchid.fasta', 'fasta'):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))

gi|2765658|emb|Z78533.1|CIZ78533
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGG...CGC')
740
gi|2765657|emb|Z78532.1|CCZ78532
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAACAG...GGC')
753
gi|2765656|emb|Z78531.1|CFZ78531
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGCAG...TAA')
748
gi|2765655|emb|Z78530.1|CMZ78530
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAAACAACAT...CAT')
744
gi|2765654|emb|Z78529.1|CLZ78529
Seq('ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAA...AAA')
733
gi|2765652|emb|Z78527.1|CYZ78527
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...CCC')
718
gi|2765651|emb|Z78526.1|CGZ78526
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...TGT')
730
gi|2765650|emb|Z78525.1|CAZ78525
Seq('TGTTGAGATAGCAGAATATACATCGAGTGAATCCGGAGGACCTGTGGTTATTCG...GCA')
704
gi|2765649|emb|Z78524.1|CFZ78524
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGATAGTAG...AGC')
740
gi|2765648|emb|Z78523.1|CHZ78523
Seq('CGTAACCAGGTTTCCGT

## Sequence Objects

A large part of the biopython infrastructure deals with tools for handling sequences. These could be DNA sequences, RNA sequences, amino acid sequences or even more exotic constructs. Generally, 'Seq' object can be treated like a normal python string. 

In [10]:
from Bio.Seq import Seq
my_seq = Seq("ACAGTAGAC")
print(my_seq)
my_seq

ACAGTAGAC


Seq('ACAGTAGAC')

If we want to code a protein sequence, we can do that just as easily.

In [11]:
my_prot = Seq("AAAAA")
my_prot

Seq('AAAAA')

We can take the length of sequences and index into them like strings.

In [12]:
print(len(my_prot))

5


In [13]:
my_prot[0]

'A'

You can also use slice notation on sequences to get subsequences.

In [14]:
my_prot[0:3]

Seq('AAA')

In [15]:
my_prot + my_prot

Seq('AAAAAAAAAA')

 Biopython automatically handles the concatenation as both sequences are of the generic alphabet.

In [16]:
my_prot + my_seq

Seq('AAAAAACAGTAGAC')

## Transcription

Transcription is the process by which a DNA sequence is converted into messenger RNA. Remember that this is part of the "central dogma" of biology in which DNA engenders messenger RNA which engenders proteins. Here's a nice representation of this cycle borrowed from a Khan academy [lesson](https://cdn.kastatic.org/ka-perseus-images/20ce29384b2e7ff0cdea72acaa5b1dbd7287ab00.png).

<img src="https://cdn.kastatic.org/ka-perseus-images/20ce29384b2e7ff0cdea72acaa5b1dbd7287ab00.png">

Note from the image above that DNA has two strands. The top strand is typically called the coding strand, and the bottom the template strand. The template strand is used for the actual transcription process of conversion into messenger RNA, but in bioinformatics, it's more common to work with the coding strand because this strand has the same sequence as the RNA transcript (except that RNA has uracil (U) instead of thymine (T)). Let's now see how we can execute a transcription computationally using Biopython.


In [17]:
from Bio.Seq import Seq
coding_dna = Seq("ATGATCTCGTAA")
print(coding_dna)

ATGATCTCGTAA


In [18]:
template_dna = coding_dna.reverse_complement()
template_dna

Seq('TTACGAGATCAT')

Note that these sequences match those in the image below. You might be confused about why the `template_dna` sequence is shown reversed. The reason is that by convention, the template strand is read in the reverse direction.

Let's now see how we can transcribe our `coding_dna` strand into messenger RNA. This will only swap 'T' for 'U' and change the alphabet for our object.

In [19]:
messenger_rna = coding_dna.transcribe()
messenger_rna

Seq('AUGAUCUCGUAA')

We can also perform a "back-transcription" to recover the original coding strand from the messenger RNA.

In [20]:
messenger_rna.back_transcribe()

Seq('ATGATCTCGTAA')

## Translation

Translation is the next step in the process, whereby a messenger RNA is transformed into a protein sequence. Here's a beautiful diagram [from Wikipedia](https://en.wikipedia.org/wiki/Translation_(biology)#/media/File:Ribosome_mRNA_translation_en.svg) that lays out the basics of this process.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/Ribosome_mRNA_translation_en.svg/1000px-Ribosome_mRNA_translation_en.svg.png">

Note how 3 nucleotides at a time correspond to one new amino acid added to the growing protein chain. A set of 3 nucleotides which codes for a given amino acid is called a "codon." We can use the `translate()` method on the messenger rna to perform this transformation in code.

messenger_rna.translate()

The translation can also be performed directly from the coding sequence DNA

In [21]:
coding_dna.translate()

Seq('MIS*')

Let's now consider a longer genetic sequence that has some more interesting structure for us to look at.

In [22]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
coding_dna.translate()

Seq('MAIVMGR*KGAR*')

In both of the sequences above, '*' represents the [stop codon](https://en.wikipedia.org/wiki/Stop_codon). A stop codon is a sequence of 3 nucleotides that turns off the protein machinery. In DNA, the stop codons are 'TGA', 'TAA', 'TAG'. Note that this latest sequence has multiple stop codons. It's possible to run the machinery up to the first stop codon and pause too.

In [23]:
coding_dna.translate(to_stop=True)

Seq('MAIVMGR')

We're going to introduce a bit of terminology here. A complete coding sequence CDS is a nucleotide sequence of messenger RNA which is made of a whole number of codons (that is, the length of the sequence is a multiple of 3), starts with a "start codon" and ends with a "stop codon". A start codon is basically the opposite of a stop codon and is mostly commonly the sequence "AUG", but can be different (especially if you're dealing with something like bacterial DNA).

Let's see how we can translate a complete CDS of bacterial messenger RNA.

In [24]:
from Bio.Seq import Seq
gene = Seq(
    "GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGGTCGCTCCCATGGCA"
    "GCACAGGCTGCGGAAATTACGTTAGTCCCGTCAGTAAAATTACAGATAGGCGATCGTGAT"
    "AATCGTGGCTATTACTGGGATGGAGGTCACTGGCGCGACCACGGCTGGTGGAAACAACAT"
    "TATGAATGGCGAGGCAATCGCTGGCACCTACACGGACCGCCGCCACCGCCGCGCCACCAT"
    "AAGAAAGCTCCTCATGATCATCACGGCGGTCATGGTCCAGGCAAACATCACCGCTAA"
)
protein_sequence = gene.translate(table="Bacterial")
print(protein_sequence)

VKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYWDGGHWRDHGWWKQHYEWRGNRWHLHGPPPPPRHHKKAPHDHHGGHGPGKHHR*


In [25]:
gene.translate(table="Bacterial", to_stop=True)

Seq('VKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYWDGGHWRDH...HHR')

# Handling Annotated Sequences

Sometimes it will be useful for us to be able to handle annotated sequences where there's richer annotations, as in GenBank or EMBL files. For these purposes, we'll want to use the `SeqRecord` class.

In [26]:
from Bio.SeqRecord import SeqRecord
help(SeqRecord)

Help on class SeqRecord in module Bio.SeqRecord:

class SeqRecord(builtins.object)
 |  SeqRecord(seq, id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=None, features=None, annotations=None, letter_annotations=None)
 |  
 |  A SeqRecord object holds a sequence and information about it.
 |  
 |  Main attributes:
 |   - id          - Identifier such as a locus tag (string)
 |   - seq         - The sequence itself (Seq object or similar)
 |  
 |  Additional attributes:
 |   - name        - Sequence name, e.g. gene name (string)
 |   - description - Additional text (string)
 |   - dbxrefs     - List of database cross references (list of strings)
 |   - features    - Any (sub)features defined (list of SeqFeature objects)
 |   - annotations - Further information about the whole sequence (dictionary).
 |     Most entries are strings, or lists of strings.
 |   - letter_annotations - Per letter/symbol annotation (restricted
 |     dictionary). This holds Pyt

Let's write a bit of code involving `SeqRecord` and see how it comes out looking.

In [27]:
from Bio.SeqRecord import SeqRecord

simple_seq = Seq("GATC")
simple_seq_r = SeqRecord(simple_seq)

In [28]:
simple_seq_r.id = "AC12345"
simple_seq_r.description = "Made up sequence"
print(simple_seq_r.id)
print(simple_seq_r.description)

AC12345
Made up sequence


Let's now see how we can use `SeqRecord` to parse a large fasta file. We'll pull down a file hosted on the biopython site.

In [29]:
!wget https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.fna

--2023-08-02 14:46:08--  https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.fna
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9853 (9.6K) [text/plain]
Saving to: ‘NC_005816.fna’

NC_005816.fna       100%[===================>]   9.62K  --.-KB/s    in 0s      

2023-08-02 14:46:08 (78.8 MB/s) - ‘NC_005816.fna’ saved [9853/9853]



In [30]:
from Bio import SeqIO

record = SeqIO.read("NC_005816.fna", "fasta")
record

SeqRecord(seq=Seq('TGTAACGAACGGTGCAATAGTGATCCACACCCAACGCCTGAAATCAGATCCAGG...CTG'), id='gi|45478711|ref|NC_005816.1|', name='gi|45478711|ref|NC_005816.1|', description='gi|45478711|ref|NC_005816.1| Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence', dbxrefs=[])

Note how there's a number of annotations attached to the `SeqRecord` object!

Let's take a closer look.

In [31]:
record.id

'gi|45478711|ref|NC_005816.1|'

In [32]:
record.name

'gi|45478711|ref|NC_005816.1|'

In [33]:
record.description

'gi|45478711|ref|NC_005816.1| Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence'

Let's now look at the same sequence, but downloaded from GenBank. We'll download the hosted file from the biopython tutorial website as before.

In [34]:
!wget https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.gb

--2023-08-02 14:46:08--  https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.gb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31838 (31K) [text/plain]
Saving to: ‘NC_005816.gb’

NC_005816.gb        100%[===================>]  31.09K  --.-KB/s    in 0.002s  

2023-08-02 14:46:08 (12.7 MB/s) - ‘NC_005816.gb’ saved [31838/31838]



In [35]:
from Bio import SeqIO

record = SeqIO.read("NC_005816.gb", "genbank")
record

SeqRecord(seq=Seq('TGTAACGAACGGTGCAATAGTGATCCACACCCAACGCCTGAAATCAGATCCAGG...CTG'), id='NC_005816.1', name='NC_005816', description='Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence', dbxrefs=['Project:58037'])

## SeqIO Objects


# .count() Method
The .count() method in Biopython's Seq object behaves similar to the .count() method of Python strings. It returns the number of non-overlapping occurrences of a specific subsequence within the sequence.

In [36]:
from Bio.Seq import Seq
my_seq = Seq("AGTACACATTG")
count_a = my_seq.count('A')
count_tg = my_seq.count('TG')
print(count_a)   # Output: 3
print(count_tg)  # Output: 1

4
1


# MutableSeq objects

Just like the normal Python string, the Seq object is “read only”, or in Python terminology, immutable. Apart from wanting the Seq object to act like a string, this is also a useful default since in many biological applications you want to ensure you are not changing your sequence data:
you can convert it into a mutable sequence (a MutableSeq object) and do pretty much anything you want with it

In [37]:
 from Bio.Seq import MutableSeq
 mutable_seq = MutableSeq("GCCATTGTAATGGGCCGCTGAAAGGGTGCCCGA")

References:
[1]https://www.khanacademy.org/science/ap-biology/gene-expression-and-regulation/transcription-and-rna-processing/a/overview-of-transcription
[2]From DNA to RNA. https://www.ncbi.nlm.nih.gov/books/NBK26887/
    